In [1]:
from cxotime import CxoTime

class UserVariables:
    "I'll do it"


user_vars = UserVariables()
user_vars.start_year = "2024"
user_vars.doy_start = "032"
user_vars.end_year = "2024"
user_vars.doy_end = "213"
user_vars.prime_ssr = "A"
user_vars.ts = CxoTime(f"{user_vars.start_year}:{user_vars.doy_start}:00:00:00")
user_vars.tp = CxoTime(f"{user_vars.end_year}:{user_vars.doy_end}:23:59:59.999")

In [2]:
# Datetimes & Days that SSR rolloved during biannual period

import warnings
warnings.filterwarnings('ignore')

from components.data_requests import ska_data_request as ska_data

if user_vars.prime_ssr == "A":
    prime, backup = "A", "B"
else:
    prime, backup = "B", "A"

print(f"Looking for when SSR-{backup} was active while SSR-{prime} was prime...")
ssr_data = ska_data(user_vars.ts, user_vars.tp, f"COS{prime}RCEN")
ssr_times, ssr_values = ssr_data.times, ssr_data.vals
ssr_not_recording_doy = []
ssr_not_recording_datetimes = {}

# Shorten list to only when SSR Prime was not recording
for index, (time, value) in enumerate(zip(ssr_times, ssr_values)):

    # Detect rollover from prime to backup
    if (ssr_values[index - 1] == "TRUE"
        and value == "FALS"
        and CxoTime(time).strftime("%j") != user_vars.doy_start
        and CxoTime(time).strftime("%j") != user_vars.doy_end
    ):
        ssr_not_recording_datetimes.setdefault("Prime to Backup",[]).append(CxoTime(time).datetime)

    # Detect rollover from backup to prime (exclude last data point)
    try:
        if (value == "FALS"
            and ssr_values[index + 1] == "TRUE"
            and CxoTime(time).strftime("%j") != user_vars.doy_start
            and CxoTime(time).strftime("%j") != user_vars.doy_end
        ):
            ssr_not_recording_datetimes.setdefault("Backup to Primary",[]).append(CxoTime(time).datetime)
    except IndexError: # drop the last data point, can't look at index+1 on last value
        pass

# Pull out DOYs backup SSR was active & remove many duplicate entries
for rollover_type, date_list in ssr_not_recording_datetimes.items():
    for date in date_list:
        ssr_not_recording_doy.append(date.strftime("%j"))

ssr_not_recording_doy = list(dict.fromkeys(ssr_not_recording_doy))


print(f"   - Days that SSR-B was active during biannual period {ssr_not_recording_doy}")

for num in range(len(ssr_not_recording_datetimes["Prime to Backup"])):
    print(num + 1, ssr_not_recording_datetimes["Prime to Backup"][num].strftime("%Y:%j:%H:%M:%S.%f"))

for num in range(len(ssr_not_recording_datetimes["Backup to Primary"])):
    print(num + 1, ssr_not_recording_datetimes["Backup to Primary"][num].strftime("%Y:%j:%H:%M:%S.%f"))


Looking for when SSR-B was active while SSR-A was prime...
   - Requesting data for MSID "COSARCEN" (2024:032:00:00:00.000 thru 2024:213:23:59:59.999)...
   - Days that SSR-B was active during biannual period ['059', '064', '075', '117', '137', '198', '207', '138', '199']
1 2024:059:00:21:43.766000
2 2024:064:09:27:04.446000
3 2024:075:05:45:51.257000
4 2024:117:05:03:08.292000
5 2024:137:19:38:17.458000
6 2024:198:19:03:39.451000
7 2024:207:07:01:01.848000
1 2024:059:04:26:03.317000
2 2024:064:22:43:13.549000
3 2024:075:12:38:08.609000
4 2024:117:05:27:09.443000
5 2024:138:03:01:58.759000
6 2024:199:01:35:05.277000
7 2024:207:17:24:16.925000


In [24]:
# Get DSN Status for Period

import openpyxl as xl
from datetime import timedelta

data_dict = {}
total_time, total_contacts = 0, 0

for month in ("February","March","April","May","June","July"):
    raw_time = timedelta(0)
    directory = (f"/share/FOT/operations/Marshall Monthly/{user_vars.start_year} Reports/"
                 f"{month}_{user_vars.start_year} Report.xlsx")
    data_per_month = xl.load_workbook(directory)

    for cell in ("G3","H3"):
        raw_time += data_per_month["Totals"][f"{cell}"].value

    per_month = {"34m month total": (raw_time.days*24 + raw_time.seconds/3600),
                 "34m contacts": data_per_month["Totals"]["B3"].value}
    data_dict.setdefault(f"{month}",per_month)

    total_time += (raw_time.days*24 + raw_time.seconds/3600)
    total_contacts += data_per_month["Totals"]["B3"].value

data_dict.setdefault("Total",{"34m month total": total_time, "34m contacts": total_contacts})


for month, data in data_dict.items():
    contacts = data_dict[f"{month}"]["34m contacts"]
    total_time = data_dict[f"{month}"]["34m month total"]

    print(f'In "{month}" there were "{contacts}" supports with a total time of "{total_time}" hours.')


In "February" there were "68" supports with a total time of "161.0" hours.
In "March" there were "81" supports with a total time of "182.83333333333334" hours.
In "April" there were "82" supports with a total time of "182.91666666666666" hours.
In "May" there were "85" supports with a total time of "191.91666666666666" hours.
In "June" there were "85" supports with a total time of "189.58333333333334" hours.
In "July" there were "86" supports with a total time of "192.08333333333334" hours.
In "Total" there were "487" supports with a total time of "1100.3333333333333" hours.


In [4]:
# Calculate mean value of MSID CSSR2CBV for entire period

print("Finding the mean value of CSSR2CBV for the biannaul period...")
data = ska_data(user_vars.ts, user_vars.tp, "CSSR2CBV", True)
values = data.vals
sum_of_values, counter = 0, 0

for value in values:
    if value != 0: # Only include values when SSR was ON.
        sum_of_values += value
        counter += 1

mean_value = sum_of_values/counter

print(f"   - The mean value for MSID CSSR2CBV was: {mean_value}")


Finding the mean value of CSSR2CBV for the biannaul period...
   - Requesting data for MSID "CSSR2CBV" (2024:032:00:00:00.000 thru 2024:213:23:59:59.999)...
   - The mean value for MSID CSSR2CBV was: 4.961542352944126


In [6]:
# Build SSR DBE & SEU by submodule plots

import plotly.graph_objects as go


def build_sbe_vs_dbe_submod_plot(user_vars):
    "Build the SBE vs DBE per submodule plot"
    base_dir = "/share/FOT/engineering/ccdm/Current_CCDM_Files/Quarterly Report/76_24Feb_24Aug"
    files = ["SBE-all-period-submod.txt","DBE-dumped-period-submod.txt"]
    data_dict = {}
    plot = go.Figure()

    def format_plot(plot):
        plot.update_yaxes(
            range=(-0.5, 18),
            constrain='domain'
        )
        plot.update_layout(
            title = {
                "text": "SBE vs DBE by Submodule\nSSR-A: Feb 2024 - Jul 2024",
                "x":0.5,
                "y":0.95,
                "xanchor":"center",
                "yanchor": "top"
            },
            font = {
                "family": "Courier New, monospace",
                "size": 14,
            },
            # plot_bgcolor="rgba(0,0,0,1)",
            # paper_bgcolor="rgba(0,0,0,1)",
            autosize=True,
            showlegend=True,
            hovermode="x unified",
            barmode = "overlay",
            legend = {
                # "bgcolor": "rgba(57,57,57,1)",
                "bordercolor": "black",
                "borderwidth": 1,
                "yanchor":"top",
                "y":0.99,
                "xanchor":"left",
                "x":0.01,
                "font":{"size":20}
            },
        )

    def add_plot_trace(plot, x, y, trace_name):
        plot.add_trace(
        go.Bar(
            x = x,
            y = y,
            name = trace_name,
        )
    )

    for file in files:
        with open(f"{base_dir}/Files/SSR/{file}") as open_file:
            for line in open_file:
                parsed = line.split()
                if "SBE" in file:
                    error_type = "SBE"
                elif "DBE" in file:
                    error_type = "DBE"
                else:
                    error_type = None
                data_dict.setdefault(f"{error_type}", []).append({int(parsed[0]):int(parsed[1])})

    sbe_x, sbe_y = [],[]
    for errors in data_dict["SBE"]:
        for submodule, sbe in errors.items():
            sbe_x.append(submodule)
            sbe_y.append(sbe)

    dbe_x, dbe_y = [],[]
    for errors in data_dict["DBE"]:
        for submodule, sbe in errors.items():
            dbe_x.append(submodule)
            dbe_y.append(sbe)

    add_plot_trace(plot, sbe_x, sbe_y, "SBE by Submodule")
    add_plot_trace(plot, dbe_x, dbe_y, "DBE by Submodule")
    format_plot(plot)
    plot.write_html(f"{base_dir}/Files/SSR/SBE_vs_DBE_by_Submodule.html")


# SSR SBE vs DBE per Submodule Data
build_sbe_vs_dbe_submod_plot(user_vars)

In [125]:
# Build SBE vs DBE by Date Plot w/ Sun Spots

import requests
import pandas as pd
import io
import datetime
# import plotly.graph_objects as go
from plotly import subplots


def add_solar_spots_data(user_vars):
    """
    Working On It
    """
    print(" - Adding Solar Spots Data...")

    def solar_spot_data_query():
        """
        Description: Build query URL from user inputs, request data from "Solar Influences 
                     Data Analysis Center Site"
        Output: Panda df of data
        """
        print("""   - Querying for Sun Spot data...""")
        query_url = "https://www.sidc.be/SILSO/INFO/sndtotcsv.php"

        while True:
            try:
                csv_data = requests.get(query_url, timeout=30).content
                break
            except TimeoutError:
                print(" - Error! Data query timed-out, trying again...")

        df = pd.read_csv(io.StringIO(
            csv_data.decode('utf-8')), header=None,
                names=["Year","Month","Day","1","Sunspot Number","2","3","4"],
                delimiter=";"
            )
        df = df.drop(columns = ["1","2","3","4"])
        data_dict = df.to_dict(orient = "list")
        return data_dict

    def format_data(data, user_vars):
        dates, sunspots = ([] for i in range(2))
        zipped_data = zip(data["Year"],data["Month"],data["Day"],data["Sunspot Number"])

        print("   - Truncating data to date range...")
        for (year,month,day,sunspot_num) in zipped_data:
            date = datetime.datetime(year,month,day)

            if user_vars.ts.datetime <= date <= user_vars.tp.datetime:
                dates.append(date)
                sunspots.append(sunspot_num)

        return dates, sunspots

    raw_data = solar_spot_data_query()
    dates, sunspots = format_data(raw_data, user_vars)

    return dates, sunspots


def format_plot(plot):
    "Format things"
    plot["layout"]["yaxis1"]["range"] = (0, 30)
    plot.update_layout(
        title = {
            "text": "SBE vs DBE by Date (SBE minus 42/104)<br>SSR-A: Feb 2024 - Jul 2024",
            "x":0.5,
            "y":0.95,
            "xanchor":"center",
            "yanchor": "top"
        },
        font = {
            "family": "Courier New, monospace",
            "size": 14,
        },
        # plot_bgcolor="rgba(0,0,0,1)",
        # paper_bgcolor="rgba(0,0,0,1)",
        autosize=True,
        showlegend=True,
        hovermode="x unified",
        barmode = "overlay",
        legend = {
            # "bgcolor": "rgba(57,57,57,1)",
            "bordercolor": "black",
            "borderwidth": 1,
            "yanchor":"top",
            "y":0.99,
            "xanchor":"left",
            "x":0.01,
            "font":{"size":20}
        },
    )


def add_plot_trace(plot, x, y, trace_name, opacity=1, secondary_y = False):
    plot.add_trace(
    go.Bar(
        x = x,
        y = y,
        name = trace_name,
        opacity = opacity,
    ),
    secondary_y = secondary_y
)


def open_txt_file(base_dir, file):
    "Open a give file by pathway, return data as a dict"
    data = []
    with open(f"{base_dir}/Files/SSR/{file}") as open_file:
        for line in open_file:
            parsed = line.split()
            date = datetime.datetime.strptime(parsed[0],"%Y%j.%H%M%S%f")

            if parsed[1] == "None":
                error = 0
            else:
                error = int(parsed[1])

            data.append([date, error])

    return data


def truncate_data(user_vars, data):
    "Truncate data to date range modules"
    return_data = []
    for data in data:
        date, error = data[0], data[1]
        
        if user_vars.ts <= date <= user_vars.tp:
            return_data.append([date, error])

    return return_data


def process_sbe_data(sbe_mod104_data, sbe_mod042_data, sbe_all_data):
    "Determine how many SBE errors actually occured in the period minus modules 104 & 42"
    corrected_data = []

    for index in range(len(sbe_all_data)):
        corrected_date = sbe_all_data[index][0]
        corrected_data_point = (
            sbe_all_data[index][1] - sbe_mod104_data[index][1] - sbe_mod042_data[index][1])
        corrected_data.append([corrected_date, corrected_data_point])

    return corrected_data


def build_sbe_vs_dbe_date_solar_plot(user_vars):
    "Build the SBE vs DBE per submodule plot"
    base_dir = "/share/FOT/engineering/ccdm/Current_CCDM_Files/Quarterly Report/76_24Feb_24Aug"
    plot = subplots.make_subplots(
        rows = 1, shared_xaxes=True, row_heights=[1],
        specs = [[{"secondary_y": True}] for i in range(1)]
        )
    # Solar Spot Data
    dates, sunspots = add_solar_spots_data(user_vars)

    # SBE Data
    sbe_mod104_data = truncate_data(user_vars, open_txt_file(base_dir, "SBE-104-mission-daily.txt"))
    sbe_mod042_data = truncate_data(user_vars, open_txt_file(base_dir, "SBE-42-mission-daily.txt"))
    sbe_all_data = truncate_data(user_vars, open_txt_file(base_dir, "SBE-all-mission-daily.txt"))
    corrected_data = process_sbe_data(sbe_mod104_data,sbe_mod042_data,sbe_all_data)

    sbe_x, sbe_y = [],[]
    for data in corrected_data:
        sbe_x.append(datetime.datetime.strptime(data[0].strftime("%Y%j"), "%Y%j"))
        sbe_y.append(data[1])

    # DBE Data
    dbe_data = open_txt_file(base_dir, "DBE-dumped-period-daily.txt")
    
    dbe_x, dbe_y = [],[]
    for data in dbe_data:
        dbe_x.append(data[0])
        dbe_y.append(data[1])

    add_plot_trace(plot, sbe_x, sbe_y, "SBE by Date")
    add_plot_trace(plot, dbe_x, dbe_y, "DBE by Date")
    add_plot_trace(plot, dates, sunspots, "Sunspots", 0.2, True)
    format_plot(plot)
    plot.write_html(f"{base_dir}/Files/SSR/SBE_vs_DBE_by_Date.html")


build_sbe_vs_dbe_date_solar_plot(user_vars)


 - Adding Solar Spots Data...
   - Querying for Sun Spot data...
   - Truncating data to date range...


In [223]:
# Generate Average SBE on Submodule 104 Plot (Used when SSR-A Was prime for the period)

import plotly.graph_objects as go
from datetime import datetime


def format_plot(plot):
    "fix the layout of things"
    plot.update_layout(
        title = {
            "text": "Average Daily SBE for SSR-A Submodule 104<br>(Aug 2012 - Jul 2024)",
            "x":0.5, "y":0.95,
            "xanchor":"center",
            "yanchor": "top"
        },
        font = {
            "family": "Courier New, monospace",
            "size": 14,
        },
        # plot_bgcolor="rgba(0,0,0,1)",
        # paper_bgcolor="rgba(0,0,0,1)",
        autosize=True,
        showlegend=True,
        hovermode="x unified",
        legend = {
            # "bgcolor": "rgba(57,57,57,1)",
            "bordercolor": "black",
            "borderwidth": 1,
            "yanchor":"top",
            "y":0.99,
            "xanchor":"left",
            "x":0.01,
            "font":{"size":20}
        },
    )
    plot.update_traces(
        marker = {"size":20}
    )
    plot.update_yaxes(title = {"text":"Average Daily SBE Count"})
    plot.update_xaxes(title = {"text":"Biannual Period"})


def add_plot_trace(plot, x, y, trace_name):
    plot.add_trace(
    go.Scatter(
        x = x,
        y = y,
        name = trace_name,
        mode = "markers"
    )
)


def open_txt_file(base_dir, file):
    "Open a give file by pathway, return data as a dict"
    data = []
    with open(f"{base_dir}/Files/SSR/{file}") as open_file:
        for line in open_file:
            parsed = line.split()
            date = datetime.strptime(parsed[0],"%Y%j.%H%M%S%f")

            if parsed[1] == "None":
                error = 0
            else:
                error = int(parsed[1])

            data.append([date, error])

    return data


def truncate_data(user_vars, data):
    "Truncate data to date range modules"
    return_data = []
    for data in data:
        date, error = data[0], data[1]
        start_date = datetime.strptime("2012:214:00:00:00", "%Y:%j:%H:%M:%S")

        if start_date <= date <= user_vars.tp:
            return_data.append([date, error])

    return return_data


def build_sbe_mod104_avg_plot(user_vars):
    "build the Average SBEs on submodule 104 plot"

    base_dir = "/share/FOT/engineering/ccdm/Current_CCDM_Files/Quarterly Report/76_24Feb_24Aug"
    plot = go.Figure()

    # Mission sbe submodule 104 data.
    sbe_mod104_data = truncate_data(user_vars, open_txt_file(base_dir, "SBE-104-mission-daily.txt"))
    
    period_range = [
        ["2012:214","2013:032"],["2013:032","2013:213"],["2013:213","2014:032"],
        ["2014:032","2014:213"],["2015:032","2015:213"],["2016:032","2016:214"],
        ["2017:032","2017:213"],["2018:032","2018:213"],["2019:032","2019:213"],
        ["2020:032","2020:214"],["2021:032","2021:213"],["2022:032","2022:213"],
        ["2023:032","2023:213"],["2024:032","2024:214"]]

    # Build averages per period
    sbe_average = []
    for period in period_range:
        count, sum_value = 0, 0
        for data in sbe_mod104_data:
                date = data[0]
                sbe  = data[1]
                period_start_date = datetime.strptime(period[0],"%Y:%j")
                period_end_date   = datetime.strptime(period[1],"%Y:%j")

                if period_start_date <= date <= period_end_date:
                    count += 1
                    sum_value += sbe

        sbe_average.append([period,sum_value/count])
    
    sbe_avg_x, sbe_avg_y = [],[]
    for data in sbe_average:
        sbe_avg_x.append(f"{data[0][0]} thru {data[0][1]}")
        sbe_avg_y.append(data[1])

    add_plot_trace(plot, sbe_avg_x, sbe_avg_y, "Average SBE for SSR-A Submodule 104")
    format_plot(plot)
    plot.write_html(f"{base_dir}/Files/SSR/Avg_SBE_Submod104.html")


build_sbe_mod104_avg_plot(user_vars)


In [23]:
from cxotime import CxoTime
from datetime import timedelta

class UserVariables:
    "I'll do it"


user_vars = UserVariables()
user_vars.start_year = "2024"
user_vars.doy_start = "032"
user_vars.end_year = "2024"
user_vars.doy_end = "213"
user_vars.prime_ssr = "A"
user_vars.ts = CxoTime(f"{user_vars.start_year}:{user_vars.doy_start}:00:00:00")
user_vars.tp = CxoTime(f"{user_vars.end_year}:{user_vars.doy_end}:23:59:59.999")

time_delta = user_vars.tp.datetime - user_vars.ts.datetime
months = []

for day in range(time_delta.days + 1):
    current_day = (user_vars.ts + timedelta(days=day)).datetime

    if current_day.strftime("%B") not in months:
        months.append(current_day.strftime("%B"))

months

['February', 'March', 'April', 'May', 'June', 'July']